In [1]:
import sys
sys.path.append("./lib/")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from lib.Methods import GeneralMethods
from lib.edasSearch import EdasHyperparameterSearch
from lib.Hiperparametros import HyperparameterSwitcher
from lib.ImportacionModelos import getClassifierNames
from lib.ImportacionModelos import getClassifierModels
from lib.ImportacionModelos import getRegressorNames
from lib.ImportacionModelos import getRegressorModels
from lib.graphicGenerator import GraphicBuilder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

__Distribución de data 80% data para entrenamiento y 20% para validación__

In [3]:
from sklearn.model_selection import KFold
seed = 9
xSize = 1055
kf = KFold(n_splits=10)
df = pd.read_csv("data/filtred.csv")
X = df[df.columns[:xSize]]
Y = df[df.columns[xSize:]]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=seed)
gbTrain = GraphicBuilder(pd.concat([X_train, y_train],axis=1))
gbTest = GraphicBuilder(pd.concat([X_test, y_test],axis=1))
# x_trainReg = pd.concat([X, Y[['BUILDINGID', 'FLOOR']]], axis=1)

## Training

In [41]:
l_building = []
l_floor = []
l_latitude = []
l_longitude = []

In [42]:
# arg1 = int(sys.argv[1]) # 0:randomized, 1:exhaustive, 2:edas, 3:eas
# arg2 = int(sys.argv[2]) # 0:FLOOR, 1:BUILDINGID, 2:LATITUDE, 3:LONGITUDE
# arg3 = int(sys.argv[3]) # 0 al 17 (classifier) 0 al 13 (regressor)
# arg4 = 0 # 1:classifier, 0: regression
#arg4 = True if arg2<=1 else False
#listProcess = ["randomized", "exhaustive", "edas", "eas"]
#listPredict = ["FLOOR", "BUILDINGID", "LATITUDE", "LONGITUDE"]


estimadorDictionary = getClassifierModels(includeEnsambled=True)
modelNameList = getClassifierNames(includeEnsambled=True)

cv = KFold(n_splits=10, shuffle=False)

from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

def accert(y_true, y_pred): 
    cm = confusion_matrix(y_true, y_pred)
    return (cm.diagonal()/cm.sum(0)).mean()

scoring_acc = {
    #'average_precision' : 'average_precision_weighted',
    #'precision': 'precision',
    #'recall': 'recall',
    #'balanced_accuracy': 'balanced_accuracy',
    #'roc_auc': 'roc_auc',
    'nbaccuracy' : make_scorer(accert),
    'accuracy': 'accuracy'
}

#"""
for idModelo in range(18):
    modelName = modelNameList[idModelo]
    estimador = estimadorDictionary[modelName]
    scoreBuilding = cross_validate(estimador, X_train, y_train.BUILDINGID, scoring=scoring_acc, cv=kf, return_train_score=False)
    scoreFloor = cross_validate(estimador, X_train, y_train.FLOOR, scoring=scoring_acc, cv=kf, return_train_score=False)
    pd.DataFrame(scoreBuilding).to_csv("result/_building/" + modelName + ".csv", index=False)
    pd.DataFrame(scoreFloor).to_csv("result/_floor/" + modelName + ".csv", index=False)
    l_building.append(scoreBuilding)
    l_floor.append(scoreFloor)
"""
idModelo = 0
modelName = modelNameList[idModelo]
# hypSwitcher = HyperparameterSwitcher(modelName)
estimador = estimadorDictionary[modelName]
# parametros = hypSwitcher.getHyperparameters()()
# searchParams = hypSwitcher.getHeurisctics()()
# estimador.fit(X_train, y_train.BUILINDINGID)
# p_building = estimador.predict(X_test)

scoreBuilding = cross_validate(estimador, X_train, y_train.BUILDINGID, scoring=scoring_acc, cv=kf, return_train_score=False)
scoreFloor = cross_validate(estimador, X_train, y_train.FLOOR, scoring=scoring_acc, cv=kf, return_train_score=False)
pd.DataFrame(scoreBuilding).to_csv("result/_building/" + modelName + ".csv", index=False)
pd.DataFrame(scoreFloor).to_csv("result/_floor/" + modelName + ".csv", index=False)
"""

'\nidModelo = 0\nmodelName = modelNameList[idModelo]\n# hypSwitcher = HyperparameterSwitcher(modelName)\nestimador = estimadorDictionary[modelName]\n# parametros = hypSwitcher.getHyperparameters()()\n# searchParams = hypSwitcher.getHeurisctics()()\n# estimador.fit(X_train, y_train.BUILINDINGID)\n# p_building = estimador.predict(X_test)\n\nscoreBuilding = cross_validate(estimador, X_train, y_train.BUILDINGID, scoring=scoring_acc, cv=kf, return_train_score=False)\nscoreFloor = cross_validate(estimador, X_train, y_train.FLOOR, scoring=scoring_acc, cv=kf, return_train_score=False)\npd.DataFrame(scoreBuilding).to_csv("result/_building/" + modelName + ".csv", index=False)\npd.DataFrame(scoreFloor).to_csv("result/_floor/" + modelName + ".csv", index=False)\n'

In [43]:
from sklearn import metrics as scoreMetrics
import geopy.distance
from functools import reduce

x_trainReg = pd.concat([X_train, y_train[['BUILDINGID']]], axis=1) # , 'FLOOR'

_meanLat = 39.9926853
_meanLon = -0.0673033
_minLongitude = -7705
_maxLongitude = -7290
_minLatitude = 4864735
_maxLatitude = 4865023
_maxLatitudeGPS = 39.993720
_maxLongitudeGPS = -0.069254
_minLatitudeGPS = 39.991626
_minLongitudeGPS = -0.065425

def longitudeToGPS(x):
    return (_maxLongitudeGPS - _minLongitudeGPS) * (x - _minLongitude) / (_maxLongitude - _minLongitude) + _minLongitudeGPS

def latitudeToGPS(x):
    return (_maxLatitudeGPS - _minLatitudeGPS) * (x - _minLatitude) / (_maxLatitude - _minLatitude) + _minLatitudeGPS

def latitudeListDistance(y_true, y_pred):
    return list(map(lambda yt,yp : geopy.distance.vincenty((_meanLon, yt),(_meanLon, yp)).m , latitudeToGPS(y_true), latitudeToGPS(y_pred)))

def longitudeListDistance(y_true, y_pred):
    return list(map(lambda yt,yp : geopy.distance.vincenty((yt, _meanLat),(yp, _meanLat)).m , longitudeToGPS(y_true), longitudeToGPS(y_pred)))
    
def distance2d(y_true, y_pred):
    ldis = []
    if ((y_true>0).sum()>0):
        #ldis = list(map(lambda yt,yp : geopy.distance.vincenty((_meanLon, yt),(_meanLon, yp)).m , latitudeToGPS(y_true), latitudeToGPS(y_pred)))
        ldis = latitudeListDistance(y_true, y_pred)
    else:
        #ldis = list(map(lambda yt,yp : geopy.distance.vincenty((yt, _meanLat),(yp, _meanLat)).m , longitudeToGPS(y_true), longitudeToGPS(y_pred)))
        ldis = longitudeListDistance(y_true, y_pred)
    return reduce(lambda x,y: x+y, ldis) / len(ldis)

def mse(y_true, y_pred):
    return scoreMetrics.mean_squared_error(y_true, y_pred)
    
def mae(y_true, y_pred):
    return scoreMetrics.mean_absolute_error(y_true, y_pred)
    
scoring_reg = {
    'mae': make_scorer(mae),# 'mean_absolute_error',
    'mse': make_scorer(mse),#'mean_squared_error',
    'distance': make_scorer(distance2d),
    'r2': 'r2'
}

estimadorDictionary = getRegressorModels(includeEnsambled=True)
modelNameList = getRegressorNames(includeEnsambled=True)


for idModelo in range(14):
    modelName = modelNameList[idModelo]
    estimador = estimadorDictionary[modelName]
    scoreLat = cross_validate(estimador, x_trainReg, y_train.LATITUDE, scoring=scoring_reg, cv=kf, return_train_score=False)
    scoreLon = cross_validate(estimador, x_trainReg, y_train.LONGITUDE, scoring=scoring_reg, cv=kf, return_train_score=False)
    pd.DataFrame(scoreLat).to_csv("result/_latitude/" + modelName + ".csv", index=False)
    pd.DataFrame(scoreLon).to_csv("result/_longitude/" + modelName + ".csv", index=False)
    l_latitude.append(scoreLat)
    l_longitude.append(scoreLon)
"""
idModelo = 0
modelName = modelNameList[idModelo]
# hypSwitcher = HyperparameterSwitcher(modelName)
estimador = estimadorDictionary[modelName]

scoreLat = cross_validate(estimador, x_trainReg, y_train.LATITUDE, scoring=scoring_reg, cv=kf, return_train_score=False)
scoreLon = cross_validate(estimador, x_trainReg, y_train.LONGITUDE, scoring=scoring_reg, cv=kf, return_train_score=False)
pd.DataFrame(scoreLat).to_csv("result/_latitude/" + modelName + ".csv", index=False)
pd.DataFrame(scoreLon).to_csv("result/_longitude/" + modelName + ".csv", index=False)
"""    

'\nidModelo = 0\nmodelName = modelNameList[idModelo]\n# hypSwitcher = HyperparameterSwitcher(modelName)\nestimador = estimadorDictionary[modelName]\n\nscoreLat = cross_validate(estimador, x_trainReg, y_train.LATITUDE, scoring=scoring_reg, cv=kf, return_train_score=False)\nscoreLon = cross_validate(estimador, x_trainReg, y_train.LONGITUDE, scoring=scoring_reg, cv=kf, return_train_score=False)\npd.DataFrame(scoreLat).to_csv("result/_latitude/" + modelName + ".csv", index=False)\npd.DataFrame(scoreLon).to_csv("result/_longitude/" + modelName + ".csv", index=False)\n'

In [36]:
pd.DataFrame(pd.DataFrame(scoreLat).mean(), columns=[modelName])

,LinearRegression
fit_time,2.272874
score_time,0.105477
test_mae,7.024801
test_mse,89.337374
test_distance,5.685750
test_r2,0.980692


In [54]:
getClassifierNames(includeEnsambled=True)

['LogisticRegression',
 'SGDClassifier',
 'PassiveAggressiveClassifier',
 'MLPClassifier',
 'LinearDiscriminantAnalysis',
 'QuadraticDiscriminantAnalysis',
 'KNeighborsClassifier',
 'DecisionTreeClassifier',
 'GaussianNB',
 'BernoulliNB',
 'MultinomialNB',
 'SVC',
 'AdaBoostClassifier',
 'GradientBoostingClassifier',
 'RandomForestClassifier',
 'ExtraTreesClassifier',
 'VotingClassifier',
 'BaggingClassifier']

In [55]:
dfLatitude = pd.DataFrame(pd.DataFrame(l_latitude[0]).mean(), columns=['LinearRegression'])
dfLatitude['Lasso'] = pd.DataFrame(pd.DataFrame(l_latitude[1]).mean())
dfLatitude['Ridge'] = pd.DataFrame(pd.DataFrame(l_latitude[2]).mean())
dfLatitude['ElasticNet'] = pd.DataFrame(pd.DataFrame(l_latitude[3]).mean())
dfLatitude['PassiveAggressiveRegressor'] = pd.DataFrame(pd.DataFrame(l_latitude[4]).mean())
dfLatitude['SVR'] = pd.DataFrame(pd.DataFrame(l_latitude[5]).mean())
dfLatitude['DecisionTreeRegressor'] = pd.DataFrame(pd.DataFrame(l_latitude[6]).mean())
dfLatitude['KNeighborsRegressor'] = pd.DataFrame(pd.DataFrame(l_latitude[7]).mean())
dfLatitude['GaussianProcessRegressor'] = pd.DataFrame(pd.DataFrame(l_latitude[8]).mean())
dfLatitude['AdaBoostRegressor'] = pd.DataFrame(pd.DataFrame(l_latitude[9]).mean())
dfLatitude['GradientBoostingRegressor'] = pd.DataFrame(pd.DataFrame(l_latitude[10]).mean())
dfLatitude['RandomForestRegressor'] = pd.DataFrame(pd.DataFrame(l_latitude[11]).mean())
dfLatitude['ExtraTreesRegressor'] = pd.DataFrame(pd.DataFrame(l_latitude[12]).mean())
dfLatitude['BaggingRegressor'] = pd.DataFrame(pd.DataFrame(l_latitude[13]).mean())
dfLatitude.to_csv("result/_latitude/Comparacion.csv", index=False)

dfLongitude = pd.DataFrame(pd.DataFrame(l_longitude[0]).mean(), columns=['LinearRegression'])
dfLongitude['Lasso'] = pd.DataFrame(pd.DataFrame(l_longitude[1]).mean())
dfLongitude['Ridge'] = pd.DataFrame(pd.DataFrame(l_longitude[2]).mean())
dfLongitude['ElasticNet'] = pd.DataFrame(pd.DataFrame(l_longitude[3]).mean())
dfLongitude['PassiveAggressiveRegressor'] = pd.DataFrame(pd.DataFrame(l_longitude[4]).mean())
dfLongitude['SVR'] = pd.DataFrame(pd.DataFrame(l_longitude[5]).mean())
dfLongitude['DecisionTreeRegressor'] = pd.DataFrame(pd.DataFrame(l_longitude[6]).mean())
dfLongitude['KNeighborsRegressor'] = pd.DataFrame(pd.DataFrame(l_longitude[7]).mean())
dfLongitude['GaussianProcessRegressor'] = pd.DataFrame(pd.DataFrame(l_longitude[8]).mean())
dfLongitude['AdaBoostRegressor'] = pd.DataFrame(pd.DataFrame(l_longitude[9]).mean())
dfLongitude['GradientBoostingRegressor'] = pd.DataFrame(pd.DataFrame(l_longitude[10]).mean())
dfLongitude['RandomForestRegressor'] = pd.DataFrame(pd.DataFrame(l_longitude[11]).mean())
dfLongitude['ExtraTreesRegressor'] = pd.DataFrame(pd.DataFrame(l_longitude[12]).mean())
dfLongitude['BaggingRegressor'] = pd.DataFrame(pd.DataFrame(l_longitude[13]).mean())
dfLongitude.to_csv("result/_longitude/Comparacion.csv", index=False)

dfFloor = pd.DataFrame(pd.DataFrame(l_floor[0]).mean(), columns=['LogisticRegression'])
dfFloor['SGDClassifier'] = pd.DataFrame(pd.DataFrame(l_floor[1]).mean())
dfFloor['PassiveAggressiveClassifier'] = pd.DataFrame(pd.DataFrame(l_floor[2]).mean())
dfFloor['MLPClassifier'] = pd.DataFrame(pd.DataFrame(l_floor[3]).mean())
dfFloor['LinearDiscriminantAnalysis'] = pd.DataFrame(pd.DataFrame(l_floor[4]).mean())
dfFloor['QuadraticDiscriminantAnalysis'] = pd.DataFrame(pd.DataFrame(l_floor[5]).mean())
dfFloor['KNeighborsClassifier'] = pd.DataFrame(pd.DataFrame(l_floor[6]).mean())
dfFloor['DecisionTreeClassifier'] = pd.DataFrame(pd.DataFrame(l_floor[7]).mean())
dfFloor['GaussianNB'] = pd.DataFrame(pd.DataFrame(l_floor[8]).mean())
dfFloor['BernoulliNB'] = pd.DataFrame(pd.DataFrame(l_floor[9]).mean())
dfFloor['MultinomialNB'] = pd.DataFrame(pd.DataFrame(l_floor[10]).mean())
dfFloor['SVC'] = pd.DataFrame(pd.DataFrame(l_floor[11]).mean())
dfFloor['AdaBoostClassifier'] = pd.DataFrame(pd.DataFrame(l_floor[12]).mean())
dfFloor['GradientBoostingClassifier'] = pd.DataFrame(pd.DataFrame(l_floor[13]).mean())
dfFloor['RandomForestClassifier'] = pd.DataFrame(pd.DataFrame(l_floor[14]).mean())
dfFloor['ExtraTreesClassifier'] = pd.DataFrame(pd.DataFrame(l_floor[15]).mean())
dfFloor['VotingClassifier'] = pd.DataFrame(pd.DataFrame(l_floor[16]).mean())
dfFloor['BaggingClassifier'] = pd.DataFrame(pd.DataFrame(l_floor[17]).mean())
dfFloor.to_csv("result/_floor/Comparacion.csv", index=False)

dfBuilding = pd.DataFrame(pd.DataFrame(l_building[0]).mean(), columns=['LogisticRegression'])
dfBuilding['SGDClassifier'] = pd.DataFrame(pd.DataFrame(l_building[1]).mean())
dfBuilding['PassiveAggressiveClassifier'] = pd.DataFrame(pd.DataFrame(l_building[2]).mean())
dfBuilding['MLPClassifier'] = pd.DataFrame(pd.DataFrame(l_building[3]).mean())
dfBuilding['LinearDiscriminantAnalysis'] = pd.DataFrame(pd.DataFrame(l_building[4]).mean())
dfBuilding['QuadraticDiscriminantAnalysis'] = pd.DataFrame(pd.DataFrame(l_building[5]).mean())
dfBuilding['KNeighborsClassifier'] = pd.DataFrame(pd.DataFrame(l_building[6]).mean())
dfBuilding['DecisionTreeClassifier'] = pd.DataFrame(pd.DataFrame(l_building[7]).mean())
dfBuilding['GaussianNB'] = pd.DataFrame(pd.DataFrame(l_building[8]).mean())
dfBuilding['BernoulliNB'] = pd.DataFrame(pd.DataFrame(l_building[9]).mean())
dfBuilding['MultinomialNB'] = pd.DataFrame(pd.DataFrame(l_building[10]).mean())
dfBuilding['SVC'] = pd.DataFrame(pd.DataFrame(l_building[11]).mean())
dfBuilding['AdaBoostClassifier'] = pd.DataFrame(pd.DataFrame(l_building[12]).mean())
dfBuilding['GradientBoostingClassifier'] = pd.DataFrame(pd.DataFrame(l_building[13]).mean())
dfBuilding['RandomForestClassifier'] = pd.DataFrame(pd.DataFrame(l_building[14]).mean())
dfBuilding['ExtraTreesClassifier'] = pd.DataFrame(pd.DataFrame(l_building[15]).mean())
dfBuilding['VotingClassifier'] = pd.DataFrame(pd.DataFrame(l_building[16]).mean())
dfBuilding['BaggingClassifier'] = pd.DataFrame(pd.DataFrame(l_building[17]).mean())
dfBuilding.to_csv("result/_building/Comparacion.csv", index=False)

#pd.DataFrame(pd.DataFrame(l_building[0]).mean(), columns=[modelName])
#l_floor
#l_latitude
#l_longitude

In [59]:
"""
# ===
'mae': make_scorer(mae),# 'mean_absolute_error',
'mse': make_scorer(mse),#'mean_squared_error',
'distance': make_scorer(distance2d),
'r2': 'r2'
# ===
'nbaccuracy' : make_scorer(accert),
'accuracy': 'accuracy'
# ===   
"""
dfLongitude

,LinearRegression,Lasso,Ridge,ElasticNet,PassiveAggressiveRegressor,SVR,DecisionTreeRegressor,KNeighborsRegressor,GaussianProcessRegressor,AdaBoostRegressor,GradientBoostingRegressor,RandomForestRegressor,ExtraTreesRegressor,BaggingRegressor
fit_time,2.531144,1.054282,0.848534,0.898072,0.783983,348.231184,1.469077,1.982650,3.064316e+02,83.009644,38.564748,8.239344,9.545212,9.469980
score_time,0.085233,0.081150,0.084967,0.084130,0.081812,154.376145,0.082450,31.751880,9.619912e+01,0.443865,0.166302,0.121974,0.135257,0.342275
test_mae,8.540289,19.371092,8.507692,32.831996,158.575941,63.791369,2.782402,3.806230,3.801109e+03,2.061294,8.871777,2.764245,2.540180,2.742543
test_mse,138.988203,552.791487,135.940109,1554.826630,36130.775816,6159.190611,56.103192,50.665884,2.235548e+07,24.625936,139.781715,30.824116,31.411138,30.021256
test_distance,8.712924,19.762664,8.679669,33.495670,161.781433,65.080863,2.838646,3.883170,3.877946e+03,2.102961,9.051113,2.820122,2.591528,2.797981
test_r2,0.991028,0.964329,0.991226,0.899752,-1.342505,0.602990,0.996383,0.996740,-1.441417e+03,0.998411,0.990982,0.998014,0.997974,0.998066


## Algorithm Modeling

__RandomizedSearch: Probando con un modelo de Clasificación, los demás se probarán en Servidor__

In [6]:
from sklearn.model_selection import RandomizedSearchCV

estimadorDictionary = getClassifierModels(includeEnsambled=True)
hypSwitcher = HyperparameterSwitcher()
process = 'randomized'
n_iteraciones = 2
idModeloPrueba = 7

result = {}
modelName = getClassifierNames(includeEnsambled=True)[idModeloPrueba]
    
estimador = estimadorDictionary[modelName]
parametros = hypSwitcher.getHyperparameters(modelName)(isDummy=False)
random_search = RandomizedSearchCV(estimador, param_distributions=parametros, 
                                   n_iter=n_iteraciones, cv=kf, scoring="accuracy", 
                                   return_train_score=False, n_jobs=-1)
random_search.fit(X_train, y_train.FLOOR)
result[modelName] = random_search.cv_results_

df1 = pd.DataFrame(np.array([result[modelName]['mean_test_score'], result[modelName]['std_test_score'],
                             result[modelName]['mean_fit_time'], result[modelName]['std_fit_time'],
                             result[modelName]['mean_score_time'], result[modelName]['std_score_time']
                            ]).T, columns = ['Accuracy', 'stdAccuracy', 'FitTime', 'stdFitTime', 'ScoreTime', 'stdScoreTime'])
df2 = pd.DataFrame(result[modelName]['params'])
dff = pd.concat([df1,df2], axis=1).sort_values(['Accuracy', 'FitTime'], ascending=[False, True])
dff.to_csv("result/" + process + "/" + modelName + ".csv", index=False)

__ExhaustiveSearch: Probando con un modelo de Clasificación, los demás se probarán en Servidor__

In [10]:
from sklearn.model_selection import GridSearchCV

process = 'exhaustive'
n_iteraciones = 2

result = {}
modelName = getClassifierNames(includeEnsambled=True)[idModeloPrueba]
    
estimador = estimadorDictionary[modelName]
parametros = hypSwitcher.getHyperparameters(modelName)(isDummy=False)
grid_search = GridSearchCV(estimador, param_grid=parametros, 
                                   cv=kf, scoring="accuracy", 
                                   return_train_score=False, n_jobs=-1)
grid_search.fit(X_train, y_train.FLOOR)
result[modelName] = grid_search.cv_results_

df1 = pd.DataFrame(np.array([result[modelName]['mean_test_score'], result[modelName]['std_test_score'],
                             result[modelName]['mean_fit_time'], result[modelName]['std_fit_time'],
                             result[modelName]['mean_score_time'], result[modelName]['std_score_time']
                            ]).T, columns = ['Accuracy', 'stdAccuracy', 'FitTime', 'stdFitTime', 'ScoreTime', 'stdScoreTime'])
df2 = pd.DataFrame(result[modelName]['params'])
dff = pd.concat([df1,df2], axis=1).sort_values(['Accuracy', 'FitTime'], ascending=[False, True])
dff.to_csv("result/" + process + "/" + modelName + ".csv", index=False)

__EdasSearch: Probando con un modelo de Clasificación, los demás se probarán en Servidor__

In [27]:
idModeloPrueba = 7
#hypSwitcher = HyperparameterSwitcher()
process = 'edas'
estimadorDictionary = getClassifierModels(includeEnsambled=True)
modelName = getClassifierNames(includeEnsambled=True)[idModeloPrueba]
estimador = estimadorDictionary[modelName]
parametros = hypSwitcher.getHyperparameters(modelName)(isDummy=False)

gm = GeneralMethods(estimador, X_train, y_train.FLOOR, seed=seed) ## manage drop duplicates in sample generation
test = EdasHyperparameterSearch(
    gm, parametros, estimador, iterations=2, sample_size=2, select_ratio=0.5, debug=True) # sample_size*select_ratio>=1
test.run()
dff = pd.DataFrame(list(test.resultados)).sort_values(['Accuracy'], ascending=False).reset_index(drop=True)
dff.to_csv("result/" + process + "/" + modelName + ".csv", index=False)

indice	Accuracy class_weight criterion
0	0.962077         None   entropy
1	0.955255         None      gini
2	0.962077         None   entropy


__EasSearch: Probando con un modelo de Clasificación, los demás se probarán en Servidor__

In [7]:
from lib.easSearch import GeneticSearchCV
idModeloPrueba = 7
estimadorDictionary = getClassifierModels(includeEnsambled=True)
hypSwitcher = HyperparameterSwitcher()
process = 'eas'

idModeloPrueba = 7
result = {}
modelName = getClassifierNames(includeEnsambled=True)[idModeloPrueba]
estimador = estimadorDictionary[modelName]
parametros = hypSwitcher.getHyperparameters(modelName)(isDummy=False)

gs2 = GeneticSearchCV(estimador, parametros, cv=kf, n_jobs=4, verbose=1, scoring='accuracy', refit=False
                     , generations_number=2, population_size=3)
result = gs2.fit(X_train, y_train.FLOOR)
dff = pd.DataFrame(list(gs2.result_cache)).sort_values(['Accuracy'], ascending=False).reset_index(drop=True)
dff.to_csv("result/" + process + "/" + modelName + ".csv", index=False)

Tipos: [1, 1], rangos: [1, 1]
--- Evolve in 4 possible combinations ---
gen	nevals	avg     	min     	max     	std       
0  	3     	0.959967	0.958082	0.962077	0.00163875
1  	2     	0.960746	0.958082	0.962077	0.0018833 
2  	2     	0.962077	0.962077	0.962077	1.11022e-16
Best individual is: {'criterion': 'entropy', 'class_weight': None}
with fitness: 0.9620774431468961


In [33]:
list(test.resultados)

[{'criterion': 'entropy',
  'class_weight': None,
  'Accuracy': 0.9620774431468961,
  'stdAccuracy': 0.005414661214317645,
  'Runtime': 2.373400092124939},
 {'criterion': 'gini',
  'class_weight': 'balanced',
  'Accuracy': 0.9597418561770128,
  'stdAccuracy': 0.0066779850547021204,
  'Runtime': 2.697361779212952},
 {'criterion': 'gini',
  'class_weight': None,
  'Accuracy': 0.9552550706822374,
  'stdAccuracy': 0.0030706812530539177,
  'Runtime': 2.5444828271865845},
 {'criterion': 'gini',
  'class_weight': None,
  'Accuracy': 0.9552550706822374,
  'stdAccuracy': 0.0030706812530539177,
  'Runtime': 2.5315757513046266}]

In [ ]:
gbTest.graphicMap3D(columns = ["LATITUDE", "LONGITUDE", "FLOOR"], filename="buildingsMap3dTest")

## Model Training

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import make_scorer
from sklearn import metrics as scoreMetrics
import geopy.distance
from functools import reduce
#from sklearn.metrics import roc_auc_score # binary
#from sklearn.metrics import auc # binary


kf = KFold(n_splits=10)
modelClassifier = RandomForestClassifier(n_jobs=-1, random_state=seed)
modelRegressor = RandomForestRegressor(n_jobs=-1, random_state=seed)


_meanLat = 39.9926853
_meanLon = -0.0673033
_minLongitude = -7705
_maxLongitude = -7290
_minLatitude = 4864735
_maxLatitude = 4865023
_maxLatitudeGPS = 39.993720
_maxLongitudeGPS = -0.069254
_minLatitudeGPS = 39.991626
_minLongitudeGPS = -0.065425

def longitudeToGPS(x):
    return (_maxLongitudeGPS - _minLongitudeGPS) * (x - _minLongitude) / (_maxLongitude - _minLongitude) + _minLongitudeGPS

def latitudeToGPS(x):
    return (_maxLatitudeGPS - _minLatitudeGPS) * (x - _minLatitude) / (_maxLatitude - _minLatitude) + _minLatitudeGPS

def latitudeListDistance(y_true, y_pred):
    return list(map(lambda yt,yp : geopy.distance.vincenty((_meanLon, yt),(_meanLon, yp)).m , latitudeToGPS(y_true), latitudeToGPS(y_pred)))

def longitudeListDistance(y_true, y_pred):
    return list(map(lambda yt,yp : geopy.distance.vincenty((yt, _meanLat),(yp, _meanLat)).m , longitudeToGPS(y_true), longitudeToGPS(y_pred)))
    
def distance2d(y_true, y_pred):
    ldis = []
    if ((y_true>0).sum()>0):
        #ldis = list(map(lambda yt,yp : geopy.distance.vincenty((_meanLon, yt),(_meanLon, yp)).m , latitudeToGPS(y_true), latitudeToGPS(y_pred)))
        ldis = latitudeListDistance(y_true, y_pred)
    else:
        #ldis = list(map(lambda yt,yp : geopy.distance.vincenty((yt, _meanLat),(yp, _meanLat)).m , longitudeToGPS(y_true), longitudeToGPS(y_pred)))
        ldis = longitudeListDistance(y_true, y_pred)
    return reduce(lambda x,y: x+y, ldis) / len(ldis)

def mse(y_true, y_pred):
    return scoreMetrics.mean_squared_error(y_true, y_pred)
    
def mae(y_true, y_pred):
    return scoreMetrics.mean_absolute_error(y_true, y_pred)
    
scoring_reg = {
    'mae': make_scorer(mae),# 'mean_absolute_error',
    'mse': make_scorer(mse),#'mean_squared_error',
    'distance': make_scorer(distance2d),
    'r2': 'r2'
}

def accert(y_true, y_pred): 
    cm = confusion_matrix(y_true, y_pred)
    return (cm.diagonal()/cm.sum(0)).mean()

scoring_acc = {
    #'average_precision' : 'average_precision_weighted',
    #'precision': 'precision',
    #'recall': 'recall',
    #'balanced_accuracy': 'balanced_accuracy',
    #'roc_auc': 'roc_auc',
    'nbaccuracy' : make_scorer(accert),
    'accuracy': 'accuracy'
}


## TODO: Add and modify some metrics
## http://scikit-learn.org/stable/modules/model_evaluation.html
## https://www.icmla-conference.org/icmla10/CFP_Tutorial_files/jose.pdf